In [1]:
# Auto Reload
%load_ext autoreload
%autoreload 2

In [2]:
# Import Modules
import csv
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import json
from pprint import pprint
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from math import log
import re
from gensim import corpora
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from urllib.parse import urljoin
from urllib.parse import urlparse

# NLTK Modules
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')
from nltk import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import chunk
from nltk.util import ngrams

# Import Custom Modules
from src.data_cleaner import *
from src.dummy_words import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wesle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wesle\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wesle\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\wesle\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [3]:
# Use lyricsgenius module
import lyricsgenius as genius
# Import Credentials
from src.credentials.keys import *

# Scrape web to get Kodak Black lyrics

In [ ]:
# Set Target Artist
trg_artist = 'Kodak Black'

# Create API Instance
api = genius.Genius(client_token)
# Search Artist Information
artist = api.search_artist(trg_artist)

Searching for songs by Kodak Black...

Song 1: "Tunnel Vision"
Song 2: "No Flockin"
Song 3: "Roll in Peace"
Song 4: "ZEZE"
Song 5: "SKRT"
Song 6: "There He Go"
Song 7: "Skrilla"
Song 8: "Codeine Dreaming"
Song 9: "Transportin’"
Song 10: "Too Many Years"
Song 11: "Like Dat"
Song 12: "Calling My Spirit"
Song 13: "I.M.Y (I Miss You)"
Song 14: "Baby Girl"
Song 15: "Institution"
Song 16: "Patty Cake"
Song 17: "Testimony"
Song 18: "Malcolm X.X.X."
Song 19: "Versatile"
Song 20: "Pimpin Ain’t Eazy"
Song 21: "G to the A"
Song 22: "Don’t Wanna Breathe"
Song 23: "Snot Thot"
Song 24: "Conscience"
Song 25: "If I’m Lyin, I’m Flyin"
Song 26: "First Day Out"
Song 27: "Everything 1K"
Song 28: "201519971800"
Song 29: "Already"
Song 30: "Water"
Song 31: "Young Prodigy"
Song 32: "Halloween"
Song 33: "Can I"
Song 34: "Gave It All I Got"
Song 35: "Expeditiously"
Song 36: "If You Ain’t Ridin’"
Song 37: "Day for Day"
Song 38: "Slayed"
Song 39: "Vibin In This Bih"
Song 40: "Honey Bun"
Song 41: "News Or Somethi

In [ ]:
# Save Lyrics from Artist
aux = artist.save_lyrics(format='json', filename=f'data/{trg_artist} Lyrics.txt', overwrite=True, skip_duplicates=True, verbose=True)